In [4]:
# Set notebook to use only one GPU
%env CUDA_VISIBLE_DEVICES=1     
%matplotlib inline

env: CUDA_VISIBLE_DEVICES=1


In [5]:
import file_functions as ff
import numpy as np
methods = ff.fetch_method_names()
per_subj_dict = ff.create_per_subj_dict()
metrics = ff.fetch_metric_names()
keys = ff.fetch_keys()      # Fetches names names for within-pop and out-of-pop sets

In [6]:
'''
Per-subject uncertainties and AUROC
This is exactly what I need:
    - Per subject AUROC. This can only be done with array of shape (9, 576).
        - start w/ (50, 9, 50, 576, 4) for a method.
        - Mean axis=0 -> (9, 50, 576, 4)
        - Get uncertainties -> (9, 576) -> Mean axis -1=Avg. uncertainties -> (9, 1)
        - For each subject in axis 0, calculate AUROC to get final array of (9, 1)
    - Array of shape (9, 1) for uncertanties
        - Get uncertainties -> (9, 576) -> Mean axis -1=Avg. uncertainties -> (9, 1)
'''

from sklearn.metrics import auc
import matplotlib.patches as mpatches
from numpy import round

'''
Gets a numpy array down to a 2D array
'''
def get_in_shape(data):
    # If data has shape of 2 elements or less, assume it's already in shape
    if len(data.shape) < 3:
        return data
    else:
        while len(data.shape) > 2:
            data = np.vstack(data)
        return data

'''
data can be whatever shape.
manual roc plot creation reqs:
    y_pred: (50, 576, 4)
'''
def get_fpr_tpr(y_true, y_pred, unc, isStandard):
    thresholds = np.arange(0, 1.001, 0.001)
    tpr = []
    fpr = []
    y_pred = get_in_shape(y_pred.mean(axis=0)) if isStandard not in [1, 2] else y_pred     # Take mean of axis with forward passes with methods aren't standard and DUQ
    y_true = get_in_shape(y_true)

    for t in thresholds:
        '''
        Order is reversed because FPR and TPR are reversed for some reason
        '''
        if isStandard == 2:     # For DUQ, AUROC is better when maximum distance to any class is chosen. But this flips the AUROC.
            certains = (t > unc)
            uncertains = (t < unc)
        else:
            certains = (t < unc)           # Certain when uncertainty is below threshold
            uncertains = (t > unc)
        # Calculate TPR and FPR
        tp = sum(y_pred.argmax(axis=1)[uncertains] != y_true.argmax(axis=1)[uncertains])     # N. preds uncertain predictions that are incorrect
        fn = sum(y_pred.argmax(axis=1)[certains] != y_true.argmax(axis=1)[certains])      # Prediction that's certain and incorrect
        fp = sum(y_pred.argmax(axis=1)[uncertains] == y_true.argmax(axis=1)[uncertains])   # prediction that's uncertain and correct
        tn = sum(y_pred.argmax(axis=1)[certains] == y_true.argmax(axis=1)[certains]) # Prediction that's certain and correct
        # print(f'tp: {tp} fn: {fn} fp: {fp} tn: {tn}')
        fpr.append(fp / (fp + tn))
        tpr.append(tp / (tp + fn))
    return np.array(tpr), np.array(fpr)

def get_auroc(y_true, y_pred, unc, isStandard):
    tpr, fpr = get_fpr_tpr(y_true, y_pred, unc, isStandard)
    return auc(tpr, fpr)

def per_subject_metrics(data, method, key, unc_method):
    key_set = data[key]        # Whether lockbox or preds of the method
    y_true = key_set['labels']
    isStandard = ff.checkIfStandard(method)
    # Average the set of 50 predictions for flipout and MC methods
    y_preds = key_set['preds'].mean(axis=0) if 'mc' in method or 'flipout' in method else key_set['preds']
    unc = ff.get_uncertainty(y_preds, unc_method, isStandard)
    per_subject_aucs = []
    for subject_id in range(y_preds.shape[0]):
        per_subject_aucs.append(get_auroc(y_true[subject_id], y_preds[subject_id], unc[subject_id], isStandard))

    return np.array(per_subject_aucs), unc.mean(axis=1)

    

'''
data: (50, 9, 50, 576, 4)
method: 'mcdropconnect'/'mcdropout'/'standard'/'standard_dropconnect'
key: 'test'/'lockbox'
'''
def do_everything(data, method, key, unc_method):
    # data shape for UQ preds: (50, 9, 50, 576, 4)
    aurocs, uncertainties = per_subject_metrics(data, method, key, unc_method)
    return aurocs, uncertainties

In [7]:
# For each method, get accuracy on test (out-of-population) 
# and lockbox (within-population) set

for method in methods:
    preds = ff.load_predictions(method)[method]
    for key in keys:
        accs = np.array(ff.get_accuracies(preds[key]))
        per_subj_dict[method]['accuracy'][key] = accs

for method in methods:
    print('-----------------------------------------------')
    print(method)
    for key in keys:
        print(key)
        data = ff.load_predictions(method)[method]
        for unc_method in metrics:
            print(unc_method)
            if unc_method == 'accuracy':
                continue
            elif unc_method == 'mutual-information' and 'standard' in method:
                continue
            aurocs, _ = ff.do_everything(data, method, key, unc_method)
            per_subj_dict[method][unc_method][key] = aurocs
            print(f'{np.mean(aurocs) * 100} +/- {np.std(aurocs) * 100}')



AttributeError: module 'file_functions' has no attribute 'get_accuracies'